### Simple option to generate parameter files

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob
from skopt.space import Space
from skopt.sampler import Lhs

In [2]:
def get_param_values(percentile,range_info):
    minval           = range_info['min']
    maxval           = range_info['max']
    value            = minval+(maxval-minval)*percentile
    
    return value

In [3]:
# Load file with parameter ranges
csv='ex_param_ranges.txt'
range_info = pd.read_csv(csv)

In [4]:
# where do you want to save the parameter files?
out_dir = './tmp/'

In [5]:
# default parameter file to start with:
basefile = '/glade/derecho/scratch/krocci/CedarCreekInputs/modified_inputs/ctsm60_params.c240208.nc'
def_paramfile = xr.open_dataset(basefile)

In [6]:
# get latin hypercube sample
n_samples = 5
n_params=3
space = Space([(0.,1.) for p in range_info.param])
lhs = Lhs(lhs_type="classic", criterion='maximin')
lhc = lhs.generate(space.dimensions, n_samples)

In [7]:
# get true parameter values
true_params = np.empty([n_samples,n_params])
for i in range(n_samples):
    true_params[i,:] = get_param_values(lhc[i],range_info)

In [8]:
# write param files
exp_name = 'lhc'
for ens in range(1,n_samples+1):
    new_paramfile = def_paramfile.copy()
    for p in range(n_params):
        param_name = range_info['param'][p]
        new_paramfile[param_name].data = true_params[ens,p]

    new_paramfile.to_netcdf(out_dir+exp_name+str(f"{ens:03}")+'.nc')
    